In [1]:
# configure train/conf.py before running this notebook
# path2code = path to the Copula-GP main dir
# path2data = path to the data dir
# path2outputs = path to the output dir, were resuls and models will be stored

In [2]:
import pickle as pkl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

import sys
home = '/home/nina/CopulaGP/'
sys.path.insert(0, home)

import torch
import bvcopula
import select_copula
import utils
import train
from vine import CVine
from train import conf

In [3]:
NSamp=10000
device = torch.device('cuda:1')
x = torch.linspace(0.,1.,NSamp).numpy()
test_x = torch.tensor(x).float().to(device=device)

In [4]:
copula_model = bvcopula.MixtureCopula(torch.linspace(0,0.99,NSamp,device=device).unsqueeze(dim=0),
                    torch.ones(1,NSamp,device=device),
                    [bvcopula.GaussianCopula_Likelihood().copula],
                    rotations=[])

In [5]:
vine = utils.get_random_vine(3,test_x,device=device)

In [15]:
copula_model01 = utils.create_model('thetas',[bvcopula.GumbelCopula_Likelihood(rotation='90°')],x.shape[0],device=device)
# copula_model02 = utils.create_model('thetas',[bvcopula.GaussianCopula_Likelihood()],x.shape[0],device=device)
# copula_model23 = utils.create_model('thetas',[bvcopula.GaussianCopula_Likelihood()],x.shape[0],device=device)
layers = [[copula_model]]#, copula_model02],[copula_model23]] # (rotation='0°'

In [16]:
vine = CVine(layers,test_x,device=device)
V = vine.sample()

In [17]:
import marginal as mg
res = np.zeros((10,2))
for i in range(10):
    V = vine.sample()
    y = V.cpu().numpy()
    res[i] = mg.revised_mi(x.reshape((*x.shape,1)),y,k=3)/np.log(2)
(res.mean(axis=0),res.std(axis=0))

(array([ 0.24461768, -0.28271219]), array([0.02466255, 0.02004706]))

In [18]:
vine.stimMI(sem_tol=0.01)

(tensor([0.1831], device='cuda:1'),
 tensor(0.0125, device='cuda:1'),
 tensor([0.2316], device='cuda:1'),
 tensor(0.0082, device='cuda:1'))

In [13]:
path = '/home/nina/CopulaGP/synthetic_data'
exp_pref='SCVine'
data = {}
y = V[0].cpu().numpy()
assert np.all(y>0) & np.all(y<1)
data['X'] = x
data['Y'] = y

with open(f"{path}/{exp_pref}_layer0.pkl",'wb') as f:
    pkl.dump(data,f)

In [ ]:
all_results = []
for layer in range(len(layers)):

    print(f'Starting {exp_pref} layer {layer}')

    results = train.train_next_layer(exp_pref, layer, device=device)
    all_results.append(results)
    with open(f"{conf.path2outputs}/{exp_pref}/models_layer{layer}.pkl","wb") as f:
        pkl.dump(results,f)
    NN = train.transform2next_layer(exp_pref,layer, device)
    print(f"NN = {NN}")

Starting SCVine layer 0
[Errno 17] File exists: '/home/nina/CopulaGP/synthetic_data/outputs/SCVine'
[Errno 17] File exists: '/home/nina/CopulaGP/synthetic_data/outputs/SCVine/layer0'
Selecting 0-1 on cpu


In [ ]:
with open(f"{path}/{exp_pref}_layer1.pkl",'rb') as f:
    layer1=pkl.load(f)

In [ ]:
plt.figure(figsize=(3,3))
colors = cm.rainbow(x)
plt.scatter(layer1['Y'][::10,0],layer1['Y'][::10,1], color=colors[::10])

In [ ]:
Y_new = np.array([V[1][...,0].cpu().numpy(),layer1['Y'][:,0]]).T

plt.figure(figsize=(3,3))
colors = cm.rainbow(x)
plt.scatter(*Y_new.T, color=colors)

In [ ]:
layers = [V[0]]
log_prob = 0
for layer, copulas in enumerate(vine.layers):
    next_layer = []
    for n, copula in enumerate(copulas):
        print(layer,layer+n+1, copula.copulas)
        log_prob += copula.log_prob(layers[-1][:,[n+1,0]])
        next_layer.append(copula.ccdf(layers[-1][:,[n+1,0]]))
    layers.append(torch.stack(next_layer,dim=-1))
print(log_prob.mean())
#         samples = torch.tensor(Y[:,[n,0]]).float().squeeze().to(device) # order!

In [ ]:
print(V[0][:,0].shape)

plt.figure(figsize=(3,3))
colors = cm.rainbow(x)
plt.scatter(V[0][:,1].cpu().numpy(),layers[2][:,0].cpu().numpy(), color=colors)

In [ ]:
V[0][...,0].shape